In [ ]:
# Databricks notebook source
# MAGIC %md
# MAGIC #  Global PE Calc

# COMMAND ----------

# MAGIC %md
# MAGIC ### library imports

# COMMAND ----------

import pandas as pd 
import numpy as np 
import time 
from currency_converter import CurrencyConverter

# COMMAND ----------

# MAGIC %md
# MAGIC ### PE Index Calculation

# COMMAND ----------

def pe_index_calc(df, equity_column_name):
    """
    This function takes the input dataframe creates the PE Index based on a chosen set of equity values. 
    For calculation on global investments, use the converted equity column (suffix -usd). For US investments only, the input 
    dataframe must be filtered for original currency in USD only.
    
    Parameters
    ----------
    df : pd.DataFrame
        DataFrame of extracted excel worksheet template data. Columns used are 'quarter', 'portfolio_company_name' 
        and an equity value column ('eq_low', 'eq_low_usd', 'eq_high', 'eq_high_usd')
    equity_column_name : str
        The equity column used to calculate the index

    Returns
    -------
    pe_index : pd.DataFrame
        DataFrame with quarterly PE Index values .
    """
    val_dates = df.Quarter.unique()

    pe_index = pd.DataFrame()


    for current_date in val_dates:    
        if 'prev_date' not in locals():
            sub_df = df[df.Quarter==current_date]

            current_companies = sub_df.portfolio_company_name
            num_companies = len(current_companies)
            remaining_companies = None
            num_remaining_companies = None
            index_equity_value_curr = sum(sub_df['eq_low'])
            index_equity_value_prev = None
            index_return = None
            index_value = 100 

            prev_date = current_date
            prev_companies = current_companies
            prev_sub_df = sub_df
            prev_index_value = index_value

            pe_index= pe_index.append({'Quarter':current_date,'NumberOfCompanies':num_companies, 'CompanyList':list(current_companies), 'NumberOfRemainingCompanies':num_remaining_companies,
                            'RemainingCompanies':remaining_companies,'IndexEquityValueCurrent': index_equity_value_curr, 'IndexEquityValuePrevious': index_equity_value_prev, 'IndexReturn': index_return, 
                            'IndexValue': index_value}, ignore_index = True)

        else: 
            sub_df = df[df.Quarter==current_date]

            current_companies = sub_df.portfolio_company_name
            num_companies = len(current_companies)
            remaining_companies = set(current_companies)& set(prev_companies)
            num_remaining_companies = len(remaining_companies)


            #index_equity_value_curr = sum(sub_df['eq_low'])
            index_equity_value_curr = sum(sub_df[df.portfolio_company_name.isin(remaining_companies)][equity_column_name])
            index_equity_value_prev = sum(prev_sub_df[df.portfolio_company_name.isin(remaining_companies)][equity_column_name])

            try:
                index_return = (index_equity_value_curr - index_equity_value_prev)/index_equity_value_prev
            except ZeroDivisionError:
                index_return = 0

            index_value = prev_index_value*(1+index_return) 


            pe_index= pe_index.append({'Quarter':current_date,'NumberOfCompanies':num_companies, 'CompanyList':list(current_companies), 'NumberOfRemainingCompanies':num_remaining_companies,
                            'RemainingCompanies':remaining_companies,'IndexEquityValueCurrent': index_equity_value_curr, 'IndexEquityValuePrevious': index_equity_value_prev, 'IndexReturn': index_return, 
                            'IndexValue': index_value}, ignore_index = True)

            prev_date = current_date
            prev_companies = current_companies
            prev_sub_df = sub_df
            prev_index_value = index_value

    #remove this
    del prev_date

    pe_index['RetentionRate'] =pe_index.NumberOfRemainingCompanies/pe_index.shift().NumberOfCompanies.replace(0, np.nan)

    return pe_index


# COMMAND ----------

# MAGIC %md
# MAGIC ##### Import data extracted from the Portfolio Valuation excel templates

# COMMAND ----------

df_global= pd.read_csv('extracted_data.csv')
df_global=df_global[df['valuation_date'].notna()]


# COMMAND ----------

df_global.columns

# COMMAND ----------

len(df_global.portfolio_company_name.unique())

# COMMAND ----------

# MAGIC %md
# MAGIC ##### To retrieve the PE Index for global companies, we need to convert any values that are not in USD

# COMMAND ----------

#convert currency to usd
c = CurrencyConverter(fallback_on_missing_rate=True)

val_date=[]
for i in df_global['valuation_date']: 
    if isinstance(i, pd._libs.tslibs.timestamps.Timestamp): 
        val_date.append(i)
    else:
        val_date.append(np.nan)
df_global['valuation_date']=val_date
df_global=df_global[df_global['valuation_date'].notna()]

rev_usd=[]
ent_low_usd=[]
ent_high_usd=[]
eq_low_usd=[]
eq_high_usd=[]
for idx, val in df_global.iterrows():
    
    try:
        ent_high_usd.append(c.convert(df_global['ent_high'][idx], df_global['currency'][idx].replace(' ',''), 'USD', date=df_global['valuation_date'][idx]))
    except: 
        #print(c.convert(df_global['ent_high'][idx], df_global['currency'][idx].replace(' ',''), 'USD', date=df_global['valuation_date'][idx]))
        ent_high_usd.append(np.nan)
    try: 
        ent_low_usd.append(c.convert(df_global['ent_low'][idx], df_global['currency'][idx].replace(' ',''), 'USD', date=df_global['valuation_date'][idx]))
    except: 
        ent_low_usd.append(np.nan)
    try:
        eq_high_usd.append(c.convert(df_global['eq_high'][idx], df_global['currency'][idx].replace(' ',''), 'USD', date=df_global['valuation_date'][idx]))
    except: 
        eq_high_usd.append(np.nan)
    try:
        eq_low_usd.append(c.convert(df_global['eq_low'][idx], df_global['currency'][idx].replace(' ',''), 'USD', date=df_global['valuation_date'][idx]))
    except: 
        eq_low_usd.append(np.nan)

df_global['ent_high_usd']=ent_high_usd
df_global['ent_low_usd']=ent_low_usd
df_global['eq_high_usd']=eq_high_usd
df_global['eq_low_usd']=eq_low_usd

# COMMAND ----------

df_global.shape

# COMMAND ----------

# MAGIC %md
# MAGIC ##### remove any rows that do not have a low equity value available

# COMMAND ----------

df_global=df_global[df_global['eq_low_usd'].notna()]

df_global = df_global.sort_values(by=['valuation_date', 'portfolio_company_name'], ignore_index=True)

# COMMAND ----------

# MAGIC %md
# MAGIC ##### Add a column with the quarters based on 'valuation_date'

# COMMAND ----------

df_global['Quarter']= pd.PeriodIndex(df_global.valuation_date, freq='Q')

pd.PeriodIndex(df_global.valuation_date, freq='Q').unique()

# COMMAND ----------

# MAGIC %md
# MAGIC ##### Calculating the PE Index
# MAGIC We are using the 'eq_low_usd' column to calculate the global PE Index. 

# COMMAND ----------

pe_index_global = pe_index_calc(df_global, 'eq_low_usd')

# COMMAND ----------

pe_index_global.to_csv('output/pe_index_global.csv', index=False)

# COMMAND ----------

# MAGIC %md
# MAGIC # Visuals 

# COMMAND ----------

pe_index_global.plot(x = 'Quarter', y='IndexValue')

# COMMAND ----------

pe_index_global.plot(x='Quarter', y='IndexReturn')

# COMMAND ----------

pe_index_global.plot(x = 'Quarter', y='RetentionRate')

# COMMAND ----------

pe_index_global.plot(x = 'Quarter', y='NumberOfCompanies') 

# COMMAND ----------

pe_index_global.plot(x = 'Quarter', y='NumberOfRemainingCompanies') 